In [1]:
!pip install tensorflow==2.2.0-rc2
!pip install tensorflow-addons

     |████████████████████████████████| 516.1 MB 23 kB/s 
     |████████████████████████████████| 2.8 MB 33.1 MB/s 
     |████████████████████████████████| 454 kB 19.8 MB/s 
     |████████████████████████████████| 777 kB 40.4 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0
     |████████████████████████████████| 1.0 MB 2.0 kB/s 


In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [3]:
import copy
import math
import os
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
import tensorflow.keras.layers as L
from tensorflow.keras import Model
from sklearn.metrics import f1_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ModelCheckpoint
import math
# from tqdm.notebook import tqdm
import gc
import os
from sklearn.metrics import recall_score, f1_score, precision_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from sklearn.model_selection import KFold
from tensorflow.python.keras.engine import training
import collections

LEN_ = 4000
n_epoch = 1000
batch_size = 64
STEP_ = LEN_
folds = 5
learning_rate = 0.001
SEED_ = 987654321
patience = 20
FOLD_ = 0
n_classes = 11

lr_schedule = LearningRateScheduler(
    PolynomialDecay(
      initial_learning_rate = learning_rate,
      decay_steps = n_epoch,
      power = 2,
      end_learning_rate = 0.0
    )
)

early_stopping = EarlyStopping(patience = patience, verbose=0)


In [4]:
class FBetaScore(tf.keras.metrics.Metric):
    """Computes F-Beta score.
    It is the weighted harmonic mean of precision
    and recall. Output range is [0, 1]. Works for
    both multi-class and multi-label classification.
    F-Beta = (1 + beta^2) * (prec * recall) / ((beta^2 * prec) + recall)
    Args:
        num_classes: Number of unique classes in the dataset.
        average: Type of averaging to be performed on data.
            Acceptable values are `None`, `micro`, `macro` and
            `weighted`. Default value is None.
        beta: Determines the weight of precision and recall
            in harmonic mean. Determines the weight given to the
            precision and recall. Default value is 1.
        threshold: Elements of `y_pred` greater than threshold are
            converted to be 1, and the rest 0. If threshold is
            None, the argmax is converted to 1, and the rest 0.
    Returns:
        F-Beta Score: float
    Raises:
        ValueError: If the `average` has values other than
        [None, micro, macro, weighted].
        ValueError: If the `beta` value is less than or equal
        to 0.
    `average` parameter behavior:
        None: Scores for each class are returned
        micro: True positivies, false positives and
            false negatives are computed globally.
        macro: True positivies, false positives and
            false negatives are computed for each class
            and their unweighted mean is returned.
        weighted: Metrics are computed for each class
            and returns the mean weighted by the
            number of true instances in each class.
    """

    def __init__(self,
                 num_classes,
                 shape = [LEN_, n_classes],
                 average=None,
                 beta=1.0,
                 threshold=None,
                 name='fbeta_score',
                 dtype=tf.float32):
        super(FBetaScore, self).__init__(name=name)

        if average not in (None, 'micro', 'macro', 'weighted'):
            raise ValueError("Unknown average type. Acceptable values "
                             "are: [None, micro, macro, weighted]")

        if not isinstance(beta, float):
            raise TypeError("The value of beta should be a python float")

        if beta <= 0.0:
            raise ValueError("beta value should be greater than zero")

        if threshold is not None:
            if not isinstance(threshold, float):
                raise TypeError(
                    "The value of threshold should be a python float")
            if threshold > 1.0 or threshold <= 0.0:
                raise ValueError("threshold should be between 0 and 1")

        self.num_classes = num_classes
        self.average = average
        self.beta = beta
        self.threshold = threshold
        self.axis = None
        self.init_shape = shape
        
        if self.average != 'micro':
            self.axis = 0
            # self.init_shape = [shape]

        def _zero_wt_init(name):
            return self.add_weight(
                name,
                shape=self.init_shape,
                initializer='zeros',
                dtype=self.dtype)

        self.true_positives = _zero_wt_init('true_positives')
        self.false_positives = _zero_wt_init('false_positives')
        self.false_negatives = _zero_wt_init('false_negatives')
        self.weights_intermediate = _zero_wt_init('weights_intermediate')

    # TODO: Add sample_weight support, currently it is
    # ignored during calculations.
    def update_state(self, y_true, y_pred, sample_weight=None):
        if self.threshold is None:
            threshold = tf.reduce_max(y_pred, axis=-1, keepdims=True)
            # make sure [0, 0, 0] doesn't become [1, 1, 1]
            # Use abs(x) > eps, instead of x != 0 to check for zero
            y_pred = tf.logical_and(y_pred >= threshold,
                                    tf.abs(y_pred) > 1e-12)
        else:
            y_pred = y_pred > self.threshold

        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)

        def _count_non_zero(val):
            non_zeros = tf.math.count_nonzero(val, axis=self.axis)
            return tf.cast(non_zeros, self.dtype)

        self.true_positives.assign_add(_count_non_zero(y_pred * y_true))
        self.false_positives.assign_add(_count_non_zero(y_pred * (y_true - 1)))
        self.false_negatives.assign_add(_count_non_zero((y_pred - 1) * y_true))
        self.weights_intermediate.assign_add(_count_non_zero(y_true))

    def result(self):
        precision = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_positives)
        recall = tf.math.divide_no_nan(
            self.true_positives, self.true_positives + self.false_negatives)

        mul_value = precision * recall
        add_value = (tf.math.square(self.beta) * precision) + recall
        mean = (tf.math.divide_no_nan(mul_value, add_value))
        f1_score = mean * (1 + tf.math.square(self.beta))

        if self.average == 'weighted':
            weights = tf.math.divide_no_nan(
                self.weights_intermediate,
                tf.reduce_sum(self.weights_intermediate))
            f1_score = tf.reduce_sum(f1_score * weights)

        elif self.average is not None:  # [micro, macro]
            f1_score = tf.reduce_mean(f1_score)

        return f1_score

    def get_config(self):
        """Returns the serializable config of the metric."""

        config = {
            "num_classes": self.num_classes,
            "average": self.average,
            "beta": self.beta,
        }

        if self.threshold is not None:
            config["threshold"] = self.threshold

        base_config = super(FBetaScore, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def reset_states(self):
        self.true_positives.assign(tf.zeros(self.init_shape, self.dtype))
        self.false_positives.assign(tf.zeros(self.init_shape, self.dtype))
        self.false_negatives.assign(tf.zeros(self.init_shape, self.dtype))
        self.weights_intermediate.assign(tf.zeros(self.init_shape, self.dtype))


In [5]:
class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

class macroF1(Callback):
    def __init__(self, model, inputs, targets, step = 10):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)
        self.step = step

    def on_train_begin(self, logs={}):    
            # Initialization code    
            self.epochs = 0    

    def on_epoch_end(self, epoch, logs):
        self.epochs += 1   
        if self.epochs % self.step == 0:  
          pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
          f1_val = f1_score(self.targets, pred, average="macro")
          print("Step: {}  --  val_f1_macro_score: {}".format(self.epochs, f1_val))

In [6]:
def get_bn_params(**params):
    axis = 2
    default_bn_params = {
        'axis': axis,
        'epsilon': 9.999999747378752e-06,
    }
    default_bn_params.update(params)
    return default_bn_params


def get_num_channels(tensor):
    channels_axis = 2
    return backend.int_shape(tensor)[channels_axis]

def get_conv_params(**params):
    default_conv_params = {
        'kernel_initializer': 'he_uniform',
        'use_bias': False,
        'padding': 'valid',
    }
    default_conv_params.update(params)
    return default_conv_params

def handle_block_names(stage, block):
    name_base = 'stage{}_unit{}_'.format(stage + 1, block + 1)
    conv_name = name_base + 'conv'
    bn_name = name_base + 'bn'
    relu_name = name_base + 'relu'
    sc_name = name_base + 'sc'
    return conv_name, bn_name, relu_name, sc_name

def expand_dims(x, channels_axis):
    return x[:, None, :]

ModelParams = collections.namedtuple(
    'ModelParams',
    ['model_name', 'repetitions', 'residual_block', 'attention']
)

def ChannelSE(reduction=16):
    """
    Squeeze and Excitation block, reimplementation inspired by
        https://github.com/Cadene/pretrained-models.pytorch/blob/master/pretrainedmodels/models/senet.py
    Args:
        reduction: channels squeeze factor
    """
#     backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    channels_axis = 2

    def layer(input_tensor):

        channels = tf.keras.backend.int_shape(input_tensor)[channels_axis]

        # get number of channels/filters
        x = input_tensor

        # squeeze and excitation block in PyTorch style with
        x = layers.GlobalAveragePooling1D()(x)
        x = layers.Lambda(expand_dims, arguments={'channels_axis': channels_axis})(x)
        x = layers.Conv1D(channels // reduction, 1, kernel_initializer='he_uniform')(x)
        x = layers.Activation('Mish')(x)
        x = layers.Conv1D(channels, 1, kernel_initializer='he_uniform')(x)
        x = layers.Activation('sigmoid')(x)

        # apply attention
        x = layers.Multiply()([input_tensor, x])

        return x

    return layer

def residual_conv_block(filters, stage, block, strides=(1, 1), attention=None, cut='pre'):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        cut: one of 'pre', 'post'. used to decide where skip connection is taken
    # Returns
        Output tensor for the block.
    """

    def layer(input_tensor):

        # get params and names of layers
        conv_params = get_conv_params()
        bn_params = get_bn_params()
        conv_name, bn_name, relu_name, sc_name = handle_block_names(stage, block)

        x = layers.BatchNormalization(name=bn_name + '1', **bn_params)(input_tensor)
        x = layers.Activation('Mish', name=relu_name + '1')(x)

        # defining shortcut connection
        if cut == 'pre':
            shortcut = input_tensor
        elif cut == 'post':
            shortcut = layers.Conv1D(filters, 1, name=sc_name, strides=strides, **conv_params)(x)
        else:
            raise ValueError('Cut type not in ["pre", "post"]')

        # continue with convolution layers
        x = layers.ZeroPadding1D(padding=1)(x)
        x = layers.Conv1D(filters, 3, strides=strides, name=conv_name + '1', **conv_params)(x)

        x = layers.BatchNormalization(name=bn_name + '2', **bn_params)(x)
        x = layers.Activation('Mish', name=relu_name + '2')(x)
        x = layers.ZeroPadding1D(padding=1)(x)
        x = layers.Conv1D(filters, 3, name=conv_name + '2', **conv_params)(x)

        # use attention block if defined
        if attention is not None:
            x = attention(x)

        # add residual connection
        x = layers.Add()([x, shortcut])
        return x

    return layer


# -------------------------------------------------------------------------
#   Residual Model Builder
# -------------------------------------------------------------------------


def ResNet(model_params, input_shape=None, include_top=False,
           classes=11, **kwargs):
    """Instantiates the ResNet, SEResNet architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    Args:
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    Returns:
        A Keras model instance.
    Raises:
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """

    input_layer = layers.Input(shape=input_shape)
    # choose residual block type
    ResidualBlock = model_params.residual_block
    if model_params.attention:
        Attention = model_params.attention(**kwargs)
    else:
        Attention = None

    # get parameters for model layers
    no_scale_bn_params = get_bn_params(scale=False)
    bn_params = get_bn_params()
    conv_params = get_conv_params()
    init_filters = 64

    # resnet bottom
    x = layers.BatchNormalization(name='bn_data', **no_scale_bn_params)(input_layer)
    x = layers.ZeroPadding1D(padding=3)(x)
    x = layers.Conv1D(init_filters, 7, strides=2, name='conv0', **conv_params)(x)
    x = layers.BatchNormalization(name='bn0', **bn_params)(x)
    x = layers.Activation('Mish', name='relu0')(x)
    x = layers.ZeroPadding1D(padding=1)(x)
    x = layers.MaxPooling1D(3, strides=2, padding='valid', name='pooling0')(x)

    # resnet body
    for stage, rep in enumerate(model_params.repetitions):
        for block in range(rep):

            filters = init_filters * (2 ** stage)

            # first block of first stage without strides because we have maxpooling before
            if block == 0 and stage == 0:
                x = ResidualBlock(filters, stage, block, strides=1,
                                  cut='post', attention=Attention)(x)

            elif block == 0:
                x = ResidualBlock(filters, stage, block, strides=2,
                                  cut='post', attention=Attention)(x)

            else:
                x = ResidualBlock(filters, stage, block, strides=1,
                                  cut='pre', attention=Attention)(x)

    x = layers.BatchNormalization(name='bn1', **bn_params)(x)
    x = layers.Activation('Mish', name='relu1')(x)

    # Create model.
    model = training.Model(input_layer, x)

    return model

def SEResNet34(input_shape= (LEN_,1), classes=11, **kwargs):
    return ResNet(
        ModelParams('seresnet34', (3, 4, 6, 3), residual_conv_block, ChannelSE),
        input_shape=input_shape,
        classes=classes,
        **kwargs
    )


In [7]:
def Conv2dBn(
        filters,
        kernel_size,
        activation='Mish',
        strides = 1,
        kernel_initializer='he_uniform',
        padding='valid'):
    """Extension of Conv2D layer with batchnorm"""

    bn_axis = 2

    def wrapper(input_tensor):

        x = layers.Conv1D(
              filters,
              kernel_size,
              strides=strides,
              padding=padding)(input_tensor)
        x = layers.BatchNormalization(axis=bn_axis)(x)
        x = layers.Activation(activation)(x)

        return x

    return wrapper

def Conv3x3BnMish(filters, use_batchnorm, name=None):

    def wrapper(input_tensor):
        return Conv2dBn(
            filters,
            kernel_size=3,
            activation='Mish',
            kernel_initializer='he_uniform',
            padding='same',
        )(input_tensor)

    return wrapper


def DecoderUpsamplingX2Block(filters, stage, use_batchnorm=False):
    up_name = 'decoder_stage{}_upsampling'.format(stage)
    conv1_name = 'decoder_stage{}a'.format(stage)
    conv2_name = 'decoder_stage{}b'.format(stage)
    concat_name = 'decoder_stage{}_concat'.format(stage)

    concat_axis = 2
    
    def wrapper(input_tensor, skip=None):
        x = layers.UpSampling1D(size=2, name=up_name)(input_tensor)

        if skip is not None:
            x = layers.Concatenate(axis=concat_axis, name=concat_name)([x, skip])

        x = Conv3x3BnMish(filters, use_batchnorm, name=conv1_name)(x)
        x = Conv3x3BnMish(filters, use_batchnorm, name=conv2_name)(x)

        return x

    return wrapper

feature_layer = ('stage4_unit1_relu1', 'stage3_unit1_relu1', 'stage2_unit1_relu1', 'relu0')

# ---------------------------------------------------------------------
#  Unet Decoder
# ---------------------------------------------------------------------

def build_unet(
        backbone,
        decoder_block = DecoderUpsamplingX2Block,
        skip_connection_layers = feature_layer,
        decoder_filters = (512, 256, 128, 64, 32, 16),
        n_upsample_blocks=5,
        classes=11,
        activation='sigmoid',
        use_batchnorm=True,
):
    input_ = backbone.input
    x = backbone.output

    # extract skip connections
    skips = ([backbone.get_layer(name=i).output if isinstance(i, str)
                  else backbone.get_layer(index=i).output for i in skip_connection_layers])
    # add center block if previous operation was maxpooling (for vgg models)
    if isinstance(backbone.layers[-1], layers.MaxPooling1D):
        x = Conv3x3BnReLU(512, use_batchnorm, name='center_block1')(x)
        x = Conv3x3BnReLU(512, use_batchnorm, name='center_block2')(x)

    # building decoder blocks
    for i in range(n_upsample_blocks):

        if i < len(skips):
            skip = skips[i]
        else:
            skip = None

        x = decoder_block(decoder_filters[i], stage=i, use_batchnorm=use_batchnorm)(x, skip)

    # model head (define number of output classes)
    x = layers.Conv1D(
        filters=classes,
        kernel_size=3,
        padding='same',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        name='final_conv',
    )(x)
    x = layers.Activation(activation, name=activation)(x)

    # create keras model instance
    model = training.Model(input_, x)

    return model

def SEResNetUnet(include_top=False, input_shape = (LEN_, 1)):
    backbone = SEResNet34()
    model = build_unet(backbone)
    model.compile(Adam(learning_rate = learning_rate), loss = [tfa.losses.SigmoidFocalCrossEntropy()],#'categorical_crossentropy', 
          metrics=[FBetaScore(n_classes, average = "macro")])

    return(model)


def load_trained_model(weights_path):
    modelf = SEResNetUnet()
    modelf.load_weights(weights_path)
    
    return(modelf)



In [8]:
model = SEResNetUnet()
print(f"Number of parameter used: {model.count_params()}")

Number of parameter used: 10292826


In [9]:
train = pd.read_csv("../input/clean-removal-of-data-drift/train.csv")
_ = pd.read_csv("../input/liverpool-ion-switching/train.csv")
train['time'] = _['time']
del _

test = pd.read_csv("../input/clean-removal-of-data-drift/test.csv")
_ = pd.read_csv("../input/liverpool-ion-switching/test.csv")
test['time'] = _['time']
del _


train_mean = train.signal.mean()
train_sigma = train.signal.std()

train['signal'] = (train['signal'] - train_mean)/train_sigma
test['signal'] = (test['signal'] - train_mean)/train_sigma

gc.collect()

1433

In [10]:
assert train.shape[0] % LEN_ == 0, 'Check dimension of LEN_ in comparison to size of train. It need to be a multiple!'

train_signal = train['signal'].values.reshape(-1, LEN_, 1)
train_target = pd.get_dummies(train['channel']).values.reshape(-1, LEN_, n_classes)

test_signal = test["signal"].values.reshape(-1,LEN_,1)


In [11]:
%%time
gc.collect()
kf = KFold(n_splits = folds, shuffle = True, random_state = SEED_)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train_signal, train_target)):
    if fold_n == FOLD_:
        print(f'BEGIN FOLD:     {fold_n} -------\n\n\n')

        model = SEResNetUnet()
        model_checkpoint = ModelCheckpoint("model" + str(fold_n) + ".hdf5",
                                           save_best_only=True, verbose=0, monitor='val_fbeta_score', mode='max')

        X_train, X_valid = train_signal[train_index,:,:], train_signal[valid_index,:,:]
        y_train, y_valid = train_target[train_index,:,:], train_target[valid_index,:,:]

        callBacks = [model_checkpoint, lr_schedule, macroF1(model, X_valid, y_valid)] #early_stopping, 

        history = model.fit(
            X_train, y_train,
            epochs = n_epoch,
            batch_size = batch_size,
            callbacks = callBacks,
            verbose = 0,
            validation_data = (X_valid, y_valid)
        )
        print(f'\n\n\nENDED FOLD:     {fold_n} -------\n\n\n')

BEGIN FOLD:     0 -------



Step: 10  --  val_f1_macro_score: 0.49456155388787976
Step: 20  --  val_f1_macro_score: 0.7066749310322747
Step: 30  --  val_f1_macro_score: 0.759301674983666
Step: 40  --  val_f1_macro_score: 0.8025299989351132
Step: 50  --  val_f1_macro_score: 0.8601029596531508
Step: 60  --  val_f1_macro_score: 0.8929693344061096
Step: 70  --  val_f1_macro_score: 0.8938505025083668
Step: 80  --  val_f1_macro_score: 0.898381203643515
Step: 90  --  val_f1_macro_score: 0.8945131938852131
Step: 100  --  val_f1_macro_score: 0.9157512979223014
Step: 110  --  val_f1_macro_score: 0.8129931776695816
Step: 120  --  val_f1_macro_score: 0.9209086649379735
Step: 130  --  val_f1_macro_score: 0.9092424851824895
Step: 140  --  val_f1_macro_score: 0.9083205288271156
Step: 150  --  val_f1_macro_score: 0.9207508352030402
Step: 160  --  val_f1_macro_score: 0.900805448164816
Step: 170  --  val_f1_macro_score: 0.9158767443673731
Step: 180  --  val_f1_macro_score: 0.9293143896105907
Step: 190 

In [12]:
%%time
# pred = np.zeros((test_signal.shape[0], test_signal.shape[1], n_classes))
# for fold_ in range(folds):
#     model = load_trained_model("model" + str(fold_) + ".hdf5")
#     pred += model.predict(test_signal)/folds
# np.save("pred.npy", pred, allow_pickle = True)

model = load_trained_model("model" + str(FOLD_) + ".hdf5")
pred = model.predict(test_signal)

np.save("pred" + str(FOLD_) + ".npy", pred, allow_pickle = True)



CPU times: user 6.78 s, sys: 264 ms, total: 7.05 s
Wall time: 7.42 s


In [13]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})

sub['open_channels'] = np.argmax(pred, axis = 2).reshape(-1)
sub.to_csv('submission.csv', index=False)